# Preperation

## Import Libraries and Data

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.dummy import DummyClassifier
from sklearn.metrics import precision_score, classification_report, plot_confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
wells4 = pd.read_csv('data/wells4.csv', index_col = 0)

## Feature Engineering

#### Create well_age feature

In [3]:
wells4['well_age'] = wells4.year_recorded - wells4.construction_year

In [4]:
print(wells4.well_age.describe())
print(wells4.well_age.loc[lambda x : x<0])

count    59400.000000
mean        15.042374
std         10.100175
min         -7.000000
25%          8.000000
50%         14.000000
75%         16.000000
max         53.000000
Name: well_age, dtype: float64
8729    -4
10441   -2
13366   -7
23373   -5
27501   -5
32619   -1
33942   -3
39559   -5
48555   -4
Name: well_age, dtype: int64


Obviously it is not possible to have a well with a negative age.  Likely erroneous values in the original dataset for construction_year or date_recorded.  Since they are few in number, I will set the negative values to null and use a SimpleImputer in the Column Transformer.

In [5]:
wells4.loc[wells4.well_age < 0, "well_age"] = np.nan

#### Convert Appropriate Numeric Columns to Categorical

In [6]:
wells4[['construction_year', 'year_recorded']] = wells4[['construction_year', 'year_recorded']].astype('str')

### Create Train Test Split

In [7]:
X = wells4.drop(['status_group'], axis=1)
y = wells4['status_group']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=52)

### Subpipes & Column Transformer

In [8]:
# Create pipelines to properly scale / encode different data types for use in column transformer

subpipe_num = Pipeline(steps=[
    ('num_impute', SimpleImputer()),
    ('ss', StandardScaler())
    ])

subpipe_cat = Pipeline(steps=[('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))])

#subpipe_ord = Pipeline(steps=[('ord', OrdinalEncoder())])

In [9]:
# Create list of features for each subpipe
cat_feat = X_train.select_dtypes(include=['object']).columns
num_feat = X_train.select_dtypes(include=['float', 'int64']).columns


In [10]:
ct = ColumnTransformer(transformers = [
    ('subpipe_num', subpipe_num, num_feat),
    ('subpipe_cat', subpipe_cat, cat_feat),
    #('subpipe_ord', subpipe_ord, ord_feat)
    ])

# Modeling
Since water is so vital, the Tanzanian government wants to focus on identifying wells that need work.  The modeling process will aim to minimize the number of wells that are predicted to be functional, but actually need work (false positives).  Therefore precision will be used as the primary scoring metric with secondary consideration for accuracy.  

### Dummy Classifier

In [11]:
# Using a pipeline to maintain consistency with later models, dummy strategy of most frequent to establlish a baseline

dummy_pipe = Pipeline(steps=[
    ('ct', ct),
    ('dum', DummyClassifier(strategy='most_frequent', random_state=52))
])

In [12]:
dummy_pipe.fit(X_train, y_train)


Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('subpipe_num',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('ss',
                                                                   StandardScaler())]),
                                                  Index(['gps_height', 'longitude', 'latitude', 'population', 'well_age'], dtype='object')),
                                                 ('subpipe_cat',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  Index(['basin', 'region', 'construction_year', 'ex

In [13]:
precision_score(y_train, dummy_pipe.predict(X_train))

0.54341189674523

Unsurprisingly, the precision score for the dummy classifier is not very good.

### First Simple Model

In [14]:
dct_pipe = Pipeline(steps=[
    ('ct',ct),
    ('dct', DecisionTreeClassifier(random_state=52))
])

In [15]:
dct_pipe.fit(X_train, y_train)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('subpipe_num',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('ss',
                                                                   StandardScaler())]),
                                                  Index(['gps_height', 'longitude', 'latitude', 'population', 'well_age'], dtype='object')),
                                                 ('subpipe_cat',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  Index(['basin', 'region', 'construction_year', 'ex

In [16]:
precision_score(y_train, dct_pipe.predict(X_train))

0.9946365211651126

In [17]:
cross_val_score(dct_pipe, X_train, y_train, scoring='precision', error_score='raise')

array([0.79621543, 0.78748718, 0.78698953, 0.79368729, 0.79621451])

#### Evaluation
The untuned decision tree model performs significantly better than the baseline, but not particularly great precision and significantly overfit. FSM was useful in identifying that some categoricals were causing it to run very slowly. Eliminated some features with further EDA and reran FSM.

### Second Model

In [18]:
lr_pipe = Pipeline(steps=[
    ('ct',ct),
    ('lr', LogisticRegression(random_state=52, max_iter=1000))
    ])

In [19]:
lr_pipe.fit(X_train, y_train)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('subpipe_num',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('ss',
                                                                   StandardScaler())]),
                                                  Index(['gps_height', 'longitude', 'latitude', 'population', 'well_age'], dtype='object')),
                                                 ('subpipe_cat',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  Index(['basin', 'region', 'construction_year', 'ex

In [20]:
# defaults and max iter=1000
cross_val_score(lr_pipe, X_train, y_train, scoring='precision', error_score='raise')

array([0.7231178 , 0.71530683, 0.71997912, 0.72472893, 0.72505699])

#### GridSearch 
Untuned LogReg underperformed it's DCT counterpart, trying Gridsearch to tune LogReg hyperparameters

In [21]:
params = {}
params['lr__solver'] = ['newton-cg', 'lbfgs', 'saga']
params['lr__C'] = [.25, .5, 1, 2]

In [22]:
gs = GridSearchCV(estimator=lr_pipe, param_grid=params, cv=5, n_jobs=-2, scoring='precision')

In [23]:
gs.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('ct',
                                        ColumnTransformer(transformers=[('subpipe_num',
                                                                         Pipeline(steps=[('num_impute',
                                                                                          SimpleImputer()),
                                                                                         ('ss',
                                                                                          StandardScaler())]),
                                                                         Index(['gps_height', 'longitude', 'latitude', 'population', 'well_age'], dtype='object')),
                                                                        ('subpipe_cat',
                                                                         Pipeline(steps=[('ohe',
                                                                               

In [24]:
gs.cv_results_['mean_test_score']

array([0.72124999, 0.72124027, 0.72128096, 0.72126113, 0.72130581,
       0.72130621, 0.72166316, 0.72163793, 0.72163713, 0.72155984,
       0.72147398, 0.72152267])

In [25]:
gs.best_params_

{'lr__C': 1, 'lr__solver': 'newton-cg'}

#### LR Follow Up GridsearchCV
Initial tuning had limited success.  Will try taking the best params and tweaking other hyperparameters

In [26]:
params = {}
params['lr__solver'] = ['newton-cg']
params['lr__C'] = [1, 2, 10]
params['lr__penalty'] = ['l1', 'l2', 'none']

In [27]:
gs1 = GridSearchCV(estimator=lr_pipe, param_grid=params, cv=5, n_jobs=-2, scoring='precision')

In [ ]:
gs1.fit(X_train, y_train)

In [ ]:
gs1.cv_results_['mean_test_score']

Upon further review, the newton-cg solver does not support l1, hence the nan score 

In [ ]:
gs1.best_params_

#### Final attempt at tuning LR

In [ ]:
params = {}
params['lr__solver'] = ['newton-cg', 'liblinear']
params['lr__C'] = [1, 10, 100]
params['lr__penalty'] = ['l2']

In [ ]:
gs1_2 = GridSearchCV(estimator=lr_pipe, param_grid=params, cv=5, n_jobs=-2, scoring='precision')
gs1_2.fit(X_train, y_train)

In [ ]:
gs1_2.best_params_

In [ ]:
gs1_2.cv_results_['mean_test_score']

#### No Improvement
With virtually no improvement after three rounds of parameter tuning, LogReg does not appear to be a good fit.

### Model 3 A Tuned DCT?

In [ ]:
gs3 = GridSearchCV(estimator=dct_pipe, param_grid=params, cv=5, n_jobs=-2, scoring='precision')

params = {}
params['dct__criterion'] = ['gini', 'entropy']
params['dct__max_depth'] = [5, 10, 25]

In [ ]:
gs3.fit(X_train, y_train)

In [ ]:
gs3.cv_results_['mean_test_score']

In [ ]:
gs3.best_params_

#### Second tune
Scores were actually down slightly from the untuned DCT and the max_depth choose the highest value, so I will try rerunning with higher values for that param

In [ ]:
gs3_2 = GridSearchCV(estimator=dct_pipe, param_grid=params, cv=5, n_jobs=-2, scoring='precision')

params = {}
params['dct__criterion'] = ['gini', 'entropy']
params['dct__max_depth'] = [25, 52, 100]

In [ ]:
gs3_2.fit(X_train, y_train)

In [ ]:
gs3_2.cv_results_['mean_test_score']

In [ ]:
gs3_2.best_params_

#### Third Tune
The best DCT params so far are gini and max_depth 25.  Will keep those and try them with other params

In [ ]:
gs3_3 = GridSearchCV(estimator=dct_pipe, param_grid=params, cv=5, n_jobs=-3, scoring='precision')

params = {}
params['dct__criterion'] = ['gini']
params['dct__max_depth'] = [25]
params['dct__min_sample_leaf'] = [5, 10, 20]
params['dct__min_impurity_decrease'] = [0, .1, 1, 5]
print(params)

In [ ]:
gs3_3.fit(X_train, y_train)

In [ ]:
gs3_3.cv_results_['mean_test_score']

In [ ]:
gs3_3.best_params_